# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [9]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
%matplotlib inline 
InteractiveShell.ast_node_interactivity = "all"

In [10]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [11]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [12]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [8]:
w = data[data.race=='w']
b = data[data.race=='b']

# Question 1

What test is appropriate for this problem? Does CLT apply?

A 2-sample t-test would be appropriate for this problem since we have a binomial population ('b' and 'w') with the standard deviation unkown. We can assume the means are normally distbriuted and the central limit theory applies since the population is greater than 30.


# Question 2

What are the null and alternate hypotheses?

Null: White and black-sounding names have the same callback rate. 

Alternative: White and black-sounding names have different callback rates.

# Question 3

In [13]:
# Separate into two datasets
w = data[data.race=='w']
b = data[data.race=='b']

# Number of resumes
n_w = len(w)
n_b = len(b)

# Proportion of callbacks
prop_w = np.sum(w['call']) / n_w
prop_b = np.sum(b['call']) / n_b
print('Percentage of callbacks for white sounding names: ', prop_w)
print('Percentage of callbacks for black sounding names: ', prop_b)

# Difference in proportion of callbacks
prop_diff = prop_w - prop_b
print('Difference in percentage of callbacks: ', prop_diff)

# T-score
t_stat, p = stats.ttest_ind(w['call'],b['call'],equal_var=False)
print('t-statistic: ', t_stat)
print('p-value: ', p)

# Standard error
s_error = np.sqrt(w['call'].var()/n_w + b['call'].var()/n_b)

# Margin of error = Critical value x Standard error of the statistic
m_error = 1.96 * s_error
print('Margin of error:', m_error)

# Confidence Interval
c_int = prop_diff + (np.array([-1, 1]) * m_error)
print('Confidence interval:', c_int)

# p-value
p_value = stats.norm.cdf(-t_stat) * 2
print('p-value:', p_value)

prop_w/prop_b


Percentage of callbacks for white sounding names:  0.09650924024640657
Percentage of callbacks for black sounding names:  0.06447638603696099
Difference in percentage of callbacks:  0.032032854209445585
t-statistic:  4.114705290861751
p-value:  3.942941513645935e-05
Margin of error: 0.015258417380692
Confidence interval: [0.01677444 0.04729127]
p-value: 3.876744018936831e-05


1.4968152866242037

In [14]:
# Frequentist Bootstrapping:
# Resample from the same population with the assumption that there is no difference between the two proportions.

df = data[['race','call']]

def get_prop_diff(sample1, sample2):
    
    prop_w = np.sum(sample1['call'] == 1)/len(sample1)
    prop_b = np.sum(sample2['call'] == 1)/len(sample2)
    
    return abs(prop_w-prop_b)
    
def get_bs_samples_diff(sample1, sample2, func, size):
    length1 = len(sample1)
    length2 = len(sample2)
    bs_prop_diffs = np.empty(size)
    
    for i in range(size):
        combined_sample = pd.concat([sample1,sample2])
        shuffled_sample = combined_sample.sample(length1+length2).reset_index(drop=True)

        new_sample1 = shuffled_sample.iloc[:length1,:]
        new_sample2 = shuffled_sample.iloc[length1:,:]
        
        bs_prop_diffs[i] = func(new_sample1,new_sample2)
        
    return bs_prop_diffs

bs_samples_diff = get_bs_samples_diff(df[df.race=='w'], df[df.race=='b'], get_prop_diff, 10000)
print(bs_samples_diff[:5])

[0.01067762 0.00328542 0.0073922  0.00328542 0.00657084]


In [18]:
# p value
p = np.sum(bs_samples_diff > prop_diff)/len(bs_samples_diff)
print(p)

0.0


Since our p-value can be approximated to 0, our sample suggests that there is an impact on how white or black sounding a name is on call back rate.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

# Question 4

Write a story describing the statistical significance in the context or the original problem.

Racial discrimination occurs throughout the world in a multitude of ways. Within the workforce, racial discrimination can take place within the work environment or even in the hiring process. This study analyzed the amount of callbacks for job applications for different  black and white sounding names in the United States. From analyzing the data, there was rate of 9.65% of callbacks for white sounding names whereas there is a 6.44% of callbacks for black sounding names. Thus for every callback for every black sounding name, a white sounding name received 1.5 times more callbacks. Using a two sample t-test and frequentist bootstrapping to analyze the data, we found the p-value for the t-test and bootstrapping are both low so we can reject the null hypothesis. This means that white and black-sounding names do not receive the same callback rate showing that race could have a significant impact on the call back rate. 

# Question 5

Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, I would not infer that race/name is the most important factor in callback success. I would suggest other variables also affect the rate of callbacks such as education, previous jobs, experience level, gaps in employment history,and other factors. We would need to study all these other factors in relation to callback success as well.